# Laboratorio 7 CartPole
## Deep Learning

- Fabiola Contreras, 22787
- Diego Duarte, 22075
- José Marchena, 22398
- Sofía Velásquez, 22049
- María José Villafuerte, 22129

### Preparar ambiente

In [4]:
import numpy as np
import gymnasium as gym
import torch
import torch.nn as nn

In [2]:
env = gym.make("CartPole-v1", render_mode="rgb_array")
env

<TimeLimit<OrderEnforcing<PassiveEnvChecker<CartPoleEnv<CartPole-v1>>>>>

### Definición de Redes

In [5]:
class QNetwork(nn.Module):
    def __init__(self, obs_size, n_actions):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(obs_size, 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, n_actions)
        )
        
    def forward(self, x):
        return self.net(x)

In [6]:
obs_size = env.observation_space.shape[0]   # 4 observaciones
n_actions = env.action_space.n              # 2 acciones

# Red en línea
online_net = QNetwork(obs_size, n_actions)

# Red de destino (clon inicial)
target_net = QNetwork(obs_size, n_actions)
target_net.load_state_dict(online_net.state_dict())  # mismos pesos al inicio
target_net.eval()  # no necesita gradientes

QNetwork(
  (net): Sequential(
    (0): Linear(in_features=4, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=128, bias=True)
    (3): ReLU()
    (4): Linear(in_features=128, out_features=2, bias=True)
  )
)